### Triple Barrier Methods

여기서는 학계 논문에서 보지 못한 대체 labeling 기법을 소개한다. 독자들이 투자 전문가라면 이 방법이 더 합리적이라는 것에 공감하리라 생각한다. 저자 Lopez de Prado는 이를 Triple Barrier라 부르는데, 세 가지 배리어 중 최초로 도달한 배리어에 따라 관측값을 labeling하기 때문이다. 첫째, 2개의 수평 배리어와 1개의 수직 배리어를 설정한다. 두 수평 배리어는 이익 실현과 손절 한도에 의해 정의되는데, 추정 변동성에 대한 동적 함수다. 세 번째 배리어는 포지션을 취한 후 지나간 바의 개수에 의해 정의된다(만기 한도). 상단 배리어에 먼저 도달하면 관측값을 1로 labeling하고, 하단 배리어에 먼저 도달하면 -1로 labeling한다. 수직 배리어에 먼저 도달하면 두 가지 선택을 할 수 있다. 바로 손익 부호로 하거나 0으로 정하는 것이다. 한도 내에서의 이익이나 손실을 실현하는 문제이므로 저자의 경우 전자를 선호하지만, 각자의 문제에 대해 0에 대한 labeling이 더 잘 작동하는지 확인하면 좋다.

In [31]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

start_date = '2000-01-01'
end_date = '2024-04-06'
ticker = 'AAPL'
data = yf.download(
    ticker, 
    start = start_date, 
    end = end_date
)

[*********************100%%**********************]  1 of 1 completed


In [32]:
from FinancialMachineLearning.filter.filter import cusum_filter

cusum_events = cusum_filter(
    data['Close'], 
    threshold = data['Close'].diff().mean() * 3, 
    time_stamps = False
)

In [33]:
cusum_events = pd.DatetimeIndex(cusum_events)

In [34]:
from FinancialMachineLearning.labeling.labeling import *
vertical_barrier = add_vertical_barrier(
    cusum_events, 
    data['Close'], 
    num_days = 7 # expariation limit
)
vertical_barrier.head()

2000-01-04   2000-01-11
2000-01-11   2000-01-18
2000-01-13   2000-01-20
2000-01-19   2000-01-26
2000-01-28   2000-02-04
Name: Date, dtype: datetime64[ns]

In [35]:
from FinancialMachineLearning.features.volatility import daily_volatility

volatility = daily_volatility(data['Close'], lookback = 60)

In [46]:
triple_barrier_events = get_events(
    close = data['Close'],
    t_events = cusum_events[1:],
    pt_sl = [2, 1], # profit taking 2, stopping loss 1
    target = volatility, # dynamic threshold
    min_ret = 0.01, # minimum position return
    num_threads = 1, # number of multi-thread 
    vertical_barrier_times = vertical_barrier, # add vertical barrier
    side_prediction = None # betting side prediction (primary model)
)
triple_barrier_events.head()

,t1,trgt,pt,sl
2000-01-11,2000-01-12,0.024043,2,1
2000-01-13,2000-01-19,0.050703,2,1
2000-01-19,2000-01-26,0.081573,2,1
2000-01-28,2000-02-04,0.069135,2,1
2000-02-07,2000-02-14,0.063289,2,1


In [40]:
labels = meta_labeling(
    triple_barrier_events, 
    data['Close']
)

In [42]:
labels.head()

,ret,trgt,bin
2000-01-11,-0.059973,0.024043,-1
2000-01-13,0.101422,0.050703,0
2000-01-19,0.034017,0.081573,0
2000-01-28,0.062731,0.069135,0
2000-02-07,0.015342,0.063289,0


In [44]:
labels['bin'].value_counts()

 0    2194
-1     818
 1     540
Name: bin, dtype: int64